**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath('C:\\Users\\Valeria\\Desktop\\ESSEC\\Electives\\Deep learning\\Assignment 3\\ffmpeg-2\\bin')
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

C:\Users\Valeria\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> Function act defines which action is taken next. Agent follows $\epsilon$-greedy policy which is an algorithm for trading off exploration and exploitation: generally agent takes best action (1 - $\epsilon$) but with small probability $\epsilon$ it explores other available options. At each step it makes the decision between random or learned action. 
The problem with pure greedy policy is that agent tries to maximize local reward and can be easily stuck in sub-optinal solution. Epsilon allows agent to explore and thus increase chances to converge to optimal path faster.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1 

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1 
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> Array $board$ represents the board for a rat and contains cells with associated rewards for different cases (neutral, cheese or the poisonous cell).

> Array $position$ contains current position of the rat (1) and also shows borders of the playground (-1). Rest elements of the array are zeroes. Current position changes after the action is taken.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Select random action
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0

        while not game_over:
            action = agent.learned_act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
          
        # Save as a mp4
        if e%10==0:
            env.draw(prefix+str(e))

        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
        
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.0/18.0. Average score (-4.0)
Win/lose count 8.5/15.0. Average score (-5.25)
Win/lose count 13.0/7.0. Average score (-1.5)
Win/lose count 14.0/15.0. Average score (-1.375)
Win/lose count 7.0/10.0. Average score (-1.7)
Win/lose count 5.5/11.0. Average score (-2.3333333333333335)
Win/lose count 12.0/12.0. Average score (-2.0)
Win/lose count 14.0/17.0. Average score (-2.125)
Win/lose count 6.5/17.0. Average score (-3.0555555555555554)
Win/lose count 9.5/14.0. Average score (-3.2)
Win/lose count 12.0/21.0. Average score (-3.727272727272727)
Win/lose count 3.5/14.0. Average score (-4.291666666666667)
Win/lose count 15.5/12.0. Average score (-3.6923076923076925)
Win/lose count 8.5/15.0. Average score (-3.892857142857143)
Win/lose count 9.0/7.0. Average score (-3.5)
Final score: -3.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> By definition the action-value function Q returns the expected cumulative reward given action and state:

$$ Q^\pi (s_t, a_t) = E[R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... | s_t, a_t] $$

> This can be further transformed:

$$ Q^\pi (s_t, a_t) = E[\sum^T_{k=0} \gamma^k R_{t+k+1} | s_t, a_t] = E_{(s',a')\sim p(.|s,a)}[r(s,a) + \sum^T_{k=1} \gamma^k R_{t+k+1} | s_t, a_t] = $$

$$ = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$

> This is called Bellman equation.

> By definition optimal policy maximizes cumulative reward:

$$ Q^* (s_t, a_t) = max_\pi Q^\pi (s_t, a_t)$$

> Applying Bellman equation to optimal policy: 

$$ Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$$ 

> Since with every update of Q-function we seek to be closer to optimal $Q^*$ the objective is defined as minimizing mean-squared error in Q-values:

$$ \mathcal{L}(\theta) = E||Q*(s,a,\theta)-Q(s,a,\theta)||^2 = $$
> using Bellman equation instead of $Q^*$:
$$ = E_{s'\sim \pi^*(.|s,a)} ||r(s,a)+\gamma\max_{a'}Q^*(s',a'\theta)-Q(s,a,\theta)||^2 = $$
> and using definition of optimal policy:
$$ E_{s'\sim \pi^*(.|s,a)} || r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta) ||^{2} $$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # add element to the memory until the buffer is full, after delete first element to add next one
        if len(self.memory) < self.max_memory:  
            self.memory.append(m)
        else:
            self.memory = self.memory[1::]
            self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))                                 #Initialize table for Q values 
                                                                                  # with all zeros
        
        for i in range(self.batch_size):
           
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()            # Get information from memory
            input_states[i] = s_                                                  # Set input state
            target_q[i] = self.model.predict(np.expand_dims(s_, axis = 0))        # Set target
            
            if game_over_:
                # If game is over ipdate Q with the last reward for action
                target_q[i][a_] = r_
            
            else:
                # If game continues update Q with discounted cumulative reward
                target_q[i][a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s_, 0)))
       
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
       
        model = Sequential()
        model.add(Flatten(input_shape = (5,5, self.n_state)))
        model.add(Dense(32, activation = 'relu'))
        #model.add(Dense(16, activation = 'relu'))
        model.add(Dense(4, activation = "linear"))                         # action size
        #print(model.summary())        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model   

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0054 | Win/lose count 6.5/6.0 (0.5)
Epoch 001/020 | Loss 0.0074 | Win/lose count 3.5/1.0 (2.5)
Epoch 002/020 | Loss 0.0343 | Win/lose count 2.5/2.0 (0.5)
Epoch 003/020 | Loss 0.0227 | Win/lose count 3.0/3.0 (0.0)
Epoch 004/020 | Loss 0.0160 | Win/lose count 5.5/3.0 (2.5)
Epoch 005/020 | Loss 0.0719 | Win/lose count 5.5/4.0 (1.5)
Epoch 006/020 | Loss 0.0059 | Win/lose count 3.5/2.0 (1.5)
Epoch 007/020 | Loss 0.0078 | Win/lose count 3.0/2.0 (1.0)
Epoch 008/020 | Loss 0.0055 | Win/lose count 3.0/1.0 (2.0)
Epoch 009/020 | Loss 0.0053 | Win/lose count 3.5/8.0 (-4.5)
Epoch 010/020 | Loss 0.0723 | Win/lose count 6.0/10.0 (-4.0)
Epoch 011/020 | Loss 0.0035 | Win/lose count 3.0/1.0 (2.0)
Epoch 012/020 | Loss 0.0092 | Win/lose count 2.5/1.0 (1.5)
Epoch 013/020 | Loss 0.0032 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/020 | Loss 0.0018 | Win/lose count 5.5/5.0 (0.5)
Epoch 015/020 | Loss 0.0094 | Win/lose count 6.0/2.0 (4.0)
Epoch 016/020 | Loss 0.0016 | Win/lose count 4.0/2.0 

#### Tune parameters

In [13]:
# Reduce learning rate to 0.05
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.05, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/020 | Loss 0.0177 | Win/lose count 1.5/7.0 (-5.5)
Epoch 001/020 | Loss 0.0052 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/020 | Loss 0.0056 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/020 | Loss 0.0094 | Win/lose count 4.5/2.0 (2.5)
Epoch 004/020 | Loss 0.0168 | Win/lose count 1.5/2.0 (-0.5)
Epoch 005/020 | Loss 0.0119 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/020 | Loss 0.0100 | Win/lose count 7.0/4.0 (3.0)
Epoch 007/020 | Loss 0.0254 | Win/lose count 2.5/4.0 (-1.5)
Epoch 008/020 | Loss 0.0102 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/020 | Loss 0.0110 | Win/lose count 2.5/1.0 (1.5)
Epoch 010/020 | Loss 0.0139 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/020 | Loss 0.0251 | Win/lose count 3.5/4.0 (-0.5)
Epoch 012/020 | Loss 0.0081 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/020 | Loss 0.0478 | Win/lose count 3.0/3.0 (0.0)
Epoch 014/020 | Loss 0.0107 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/020 | Loss 0.0522 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/020 | Loss 0.0094 | Win/lose count 5.0/7.0

In [14]:
# Increase learning rate to 0.25
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.25, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/020 | Loss 0.0021 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/020 | Loss 0.0103 | Win/lose count 1.5/2.0 (-0.5)
Epoch 002/020 | Loss 0.0162 | Win/lose count 2.0/9.0 (-7.0)
Epoch 003/020 | Loss 0.0682 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/020 | Loss 0.0092 | Win/lose count 3.0/4.0 (-1.0)
Epoch 005/020 | Loss 0.0694 | Win/lose count 3.5/4.0 (-0.5)
Epoch 006/020 | Loss 0.0024 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/020 | Loss 0.0111 | Win/lose count 3.5/3.0 (0.5)
Epoch 008/020 | Loss 0.0364 | Win/lose count 3.0/3.0 (0.0)
Epoch 009/020 | Loss 0.0017 | Win/lose count 2.0/2.0 (0.0)
Epoch 010/020 | Loss 0.0014 | Win/lose count 5.0/4.0 (1.0)
Epoch 011/020 | Loss 0.0623 | Win/lose count 7.0/1.0 (6.0)
Epoch 012/020 | Loss 0.0585 | Win/lose count 5.0/0 (5.0)
Epoch 013/020 | Loss 0.0085 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/020 | Loss 0.0045 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/020 | Loss 0.0042 | Win/lose count 2.0/2.0 (0.0)
Epoch 016/020 | Loss 0.0061 | Win/lose count 2.5/4.0

In [15]:
# Increase batch size to 50
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 50)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/020 | Loss 0.0121 | Win/lose count 5.5/2.0 (3.5)
Epoch 001/020 | Loss 0.0097 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/020 | Loss 0.0560 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/020 | Loss 0.0051 | Win/lose count 3.5/2.0 (1.5)
Epoch 004/020 | Loss 0.0321 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/020 | Loss 0.0045 | Win/lose count 3.0/4.0 (-1.0)
Epoch 006/020 | Loss 0.0022 | Win/lose count 4.0/1.0 (3.0)
Epoch 007/020 | Loss 0.0145 | Win/lose count 5.0/1.0 (4.0)
Epoch 008/020 | Loss 0.0038 | Win/lose count 3.5/0 (3.5)
Epoch 009/020 | Loss 0.0077 | Win/lose count 2.0/2.0 (0.0)
Epoch 010/020 | Loss 0.0049 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/020 | Loss 0.0022 | Win/lose count 2.5/4.0 (-1.5)
Epoch 012/020 | Loss 0.0039 | Win/lose count 6.5/5.0 (1.5)
Epoch 013/020 | Loss 0.0064 | Win/lose count 5.0/2.0 (3.0)
Epoch 014/020 | Loss 0.0045 | Win/lose count 5.0/2.0 (3.0)
Epoch 015/020 | Loss 0.0058 | Win/lose count 3.0/0 (3.0)
Epoch 016/020 | Loss 0.0110 | Win/lose count 2.5/3.0 (-0.

In [16]:
# Increase memory size to 3000
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=3000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/020 | Loss 0.0172 | Win/lose count 3.5/2.0 (1.5)
Epoch 001/020 | Loss 0.0224 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/020 | Loss 0.0072 | Win/lose count 2.0/5.0 (-3.0)
Epoch 003/020 | Loss 0.0467 | Win/lose count 6.5/2.0 (4.5)
Epoch 004/020 | Loss 0.0074 | Win/lose count 2.0/8.0 (-6.0)
Epoch 005/020 | Loss 0.0025 | Win/lose count 5.0/9.0 (-4.0)
Epoch 006/020 | Loss 0.0032 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/020 | Loss 0.0066 | Win/lose count 3.0/2.0 (1.0)
Epoch 008/020 | Loss 0.0049 | Win/lose count 2.0/6.0 (-4.0)
Epoch 009/020 | Loss 0.0126 | Win/lose count 3.0/1.0 (2.0)
Epoch 010/020 | Loss 0.0077 | Win/lose count 4.5/3.0 (1.5)
Epoch 011/020 | Loss 0.1145 | Win/lose count 1.0/2.0 (-1.0)
Epoch 012/020 | Loss 0.0180 | Win/lose count 3.5/1.0 (2.5)
Epoch 013/020 | Loss 0.0053 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/020 | Loss 0.0065 | Win/lose count 0.5/3.0 (-2.5)
Epoch 015/020 | Loss 0.0054 | Win/lose count 1.5/3.0 (-1.5)
Epoch 016/020 | Loss 0.0089 | Win/lose count 5.0

In [17]:
# Decrease memory size to 1000
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/020 | Loss 0.0162 | Win/lose count 5.0/8.0 (-3.0)
Epoch 001/020 | Loss 0.0279 | Win/lose count 4.0/6.0 (-2.0)
Epoch 002/020 | Loss 0.0071 | Win/lose count 4.0/1.0 (3.0)
Epoch 003/020 | Loss 0.0030 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/020 | Loss 0.0587 | Win/lose count 4.5/1.0 (3.5)
Epoch 005/020 | Loss 0.0095 | Win/lose count 2.0/1.0 (1.0)
Epoch 006/020 | Loss 0.0039 | Win/lose count 1.5/3.0 (-1.5)
Epoch 007/020 | Loss 0.0028 | Win/lose count 4.0/6.0 (-2.0)
Epoch 008/020 | Loss 0.0185 | Win/lose count 4.5/7.0 (-2.5)
Epoch 009/020 | Loss 0.0064 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/020 | Loss 0.0075 | Win/lose count 3.5/3.0 (0.5)
Epoch 011/020 | Loss 0.0019 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/020 | Loss 0.0025 | Win/lose count 3.0/4.0 (-1.0)
Epoch 013/020 | Loss 0.0057 | Win/lose count 4.5/6.0 (-1.5)
Epoch 014/020 | Loss 0.0071 | Win/lose count 3.5/4.0 (-0.5)
Epoch 015/020 | Loss 0.0077 | Win/lose count 3.0/4.0 (-1.0)
Epoch 016/020 | Loss 0.0085 | Win/lose count 2

> Observations: 
 - loss oscillates: it tends to decrease but there are some spikes
 - increasing butch size and memory size can improve the performance (opposite is also true)
 - increasing learning rate doesn't increase performance.
 
> Best results were given by model with following parameters: lr=.1, memory_size=2000, batch_size = 50.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [18]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation= 'relu' ,input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, (3, 3), activation= 'relu'))
        model.add(Flatten())
        #model.add(Dense(16, activation = 'relu'))
        model.add(Dense(4, activation='softmax'))
        #print(model.summary())    
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [19]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0022 | Win/lose count 4.0/1.0 (3.0)
Epoch 001/020 | Loss 0.0018 | Win/lose count 1.5/4.0 (-2.5)
Epoch 002/020 | Loss 0.0084 | Win/lose count 3.5/6.0 (-2.5)
Epoch 003/020 | Loss 0.0040 | Win/lose count 5.0/8.0 (-3.0)
Epoch 004/020 | Loss 0.0000 | Win/lose count 6.5/3.0 (3.5)
Epoch 005/020 | Loss 0.0094 | Win/lose count 8.0/6.0 (2.0)
Epoch 006/020 | Loss 0.0092 | Win/lose count 2.0/5.0 (-3.0)
Epoch 007/020 | Loss 0.0175 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/020 | Loss 0.0097 | Win/lose count 6.0/1.0 (5.0)
Epoch 009/020 | Loss 0.0042 | Win/lose count 4.0/1.0 (3.0)
Epoch 010/020 | Loss 0.0133 | Win/lose count 3.5/5.0 (-1.5)
Epoch 011/020 | Loss 0.0002 | Win/lose count 4.5/1.0 (3.5)
Epoch 012/020 | Loss 0.0058 | Win/lose count 4.0/6.0 (-2.0)
Epoch 013/020 | Loss 0.0037 | Win/lose count 1.0/5.0 (-4.0)
Epoch 014/020 | Loss 0.0001 | Win/lose count 3.5/4.0 (-0.5)
Epoch 015/020 | Loss 0.0091 | Win/lose count 1.5/0 (1.5)
Epoch 016/020 | Loss 0.0197 | Win/lose count 4.5/4

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.5/1.0. Average score (1.0)
Win/lose count 2.5/4.0. Average score (0.16666666666666666)
Win/lose count 3.0/0. Average score (0.875)
Win/lose count 0.5/0. Average score (0.8)
Win/lose count 3.5/1.0. Average score (1.0833333333333333)
Win/lose count 1.0/0. Average score (1.0714285714285714)
Win/lose count 3.0/0. Average score (1.3125)
Win/lose count 4.0/0. Average score (1.6111111111111112)
Win/lose count 1.5/0. Average score (1.6)
Win/lose count 5.0/0. Average score (1.9090909090909092)
Win/lose count 1.5/2.0. Average score (1.7083333333333333)
Win/lose count 1.0/0. Average score (1.6538461538461537)
Win/lose count 4.5/2.0. Average score (1.7142857142857142)
Win/lose count 1.5/0. Average score (1.7)
Final score: 1.7
Test of the FC
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 4.0/0. Average score (3.25)
Win/lose count 0/0. Average score (2.1666666666666665)
Win/lose count 1.5/0. Average score (2.0)
Win/

In [21]:
HTML(display_videos('cnn_test10.mp4'))

In [22]:
HTML(display_videos('fc_test10.mp4'))

> Observations:
 - In general CNN performes better then FC network.
 - Agent mostly moves around its current position when there are no more "food". Rat prefers to go to empty cell rather than eat poision

#### Experiments with temperature

In [23]:
temp_test = [0.1, 0.3, 0.5, 0.7, 0.9]
for temp in temp_test:
    print ('temperature =', temp)
   
    env = Environment(grid_size=size, max_time=T,temperature=temp)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
    print('Test of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('Test of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')

temperature = 0.1
Test of the CNN
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 2.0/1.0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 2.5/0. Average score (1.125)
Win/lose count 0.5/1.0. Average score (0.8)
Win/lose count 2.0/0. Average score (1.0)
Win/lose count 1.5/0. Average score (1.0714285714285714)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 1.0/1.0. Average score (0.8888888888888888)
Win/lose count 0.5/0. Average score (0.85)
Win/lose count 1.0/0. Average score (0.8636363636363636)
Win/lose count 1.0/0. Average score (0.875)
Win/lose count 0/1.0. Average score (0.7307692307692307)
Win/lose count 0.5/0. Average score (0.7142857142857143)
Win/lose count 0.5/0. Average score (0.7)
Final score: 0.7
Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/0. Average score (0.25)
Win/lose count 0/0. Average score (0.16666666666666666)
Win/lose count 1.0/0. Average score (0.375)
Win/lose count 1.5/

> Observations:
- Increasing temperature generally improves performance. But increase in the score happens due to increase in probability of a "cheese" cell rather than poisonious cells
- CNN is more sensitive to changes in temperature showing increase in score. However, with high temperature 1-layer FC outperformes CNN.   


> | Temperature  |  0.1  |  0.3  |  0.5  |  0.7  |  0.9 |
  |--------------|-------|-------|-------|-------|------|
  | CNN          |  0.7  |  2.0  |  3.2  |  4.6  |  5.6 |
  | FC           |  0.5  |  1.0  |  1.5  |  2.9  |  7.7 |


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [24]:
def train_explore(agent, env, epoch, decay_rate = 0.96, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(decay_rate*agent.epsilon)  # epsilon will decrease by decay rate
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))   # add for previously visited position

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1 

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:                                           # the reward is given only when trained, for test real rewards are kept  
            reward = -self.malus_position[self.x, self.y]   # reduce reward for visiting same cells
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2 +> we add malus position to states
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.malus_position[self.x, self.y] = 0.1

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1 
        self.board[self.x,self.y] = 0
        self.t = 0

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [25]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0019 | Win/lose count 3.0/21.70000000000005 (-18.70000000000005)
Epoch 001/020 | Loss 0.0171 | Win/lose count 9.0/26.400000000000077 (-17.400000000000077)
Epoch 002/020 | Loss 0.0169 | Win/lose count 3.0/21.40000000000004 (-18.40000000000004)
Epoch 003/020 | Loss 0.0180 | Win/lose count 7.5/19.70000000000002 (-12.20000000000002)
Epoch 004/020 | Loss 0.0107 | Win/lose count 3.0/22.60000000000006 (-19.60000000000006)
Epoch 005/020 | Loss 0.0193 | Win/lose count 10.0/21.200000000000028 (-11.200000000000028)
Epoch 006/020 | Loss 0.0056 | Win/lose count 6.0/26.30000000000006 (-20.30000000000006)
Epoch 007/020 | Loss 0.0117 | Win/lose count 12.5/20.100000000000026 (-7.600000000000026)
Epoch 008/020 | Loss 0.0037 | Win/lose count 6.5/17.999999999999986 (-11.499999999999986)
Epoch 009/020 | Loss 0.0016 | Win/lose count 7.0/17.29999999999998 (-10.29999999999998)
Epoch 010/020 | Loss 0.0207 | Win/lose count 8.5/19.199999999999996 (-10.699999999999996)
Epoch 011/020 | Loss 

In [26]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 2.0/0. Average score (2.0)
Win/lose count 14.5/0. Average score (8.25)
Win/lose count 3.0/0. Average score (6.5)
Win/lose count 8.5/1.0. Average score (6.75)
Win/lose count 8.0/1.0. Average score (6.8)
Win/lose count 11.5/0. Average score (7.583333333333333)
Win/lose count 7.5/1.0. Average score (7.428571428571429)
Win/lose count 3.5/0. Average score (6.9375)
Win/lose count 7.0/0. Average score (6.944444444444445)
Win/lose count 5.5/2.0. Average score (6.6)
Win/lose count 6.5/1.0. Average score (6.5)
Win/lose count 5.0/2.0. Average score (6.208333333333333)
Win/lose count 5.5/1.0. Average score (6.076923076923077)
Win/lose count 1.5/0. Average score (5.75)
Win/lose count 4.0/0. Average score (5.633333333333334)
Final score: 5.633333333333334


> Decay epsilon-greedy helps to have better training since agent make less mistakes because of random actions. Thus, it makes exploration more carefull.

> What encourages more efficient exploration is penalty for previously visited cells. Rat gets less stuck in around its position but it still happens.

> Therefore, optimizing exploration strategy helps to get better performance (more winning than loosing) but it doesn't increase score dramatically comparing to previous models.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***